In [36]:
import numpy as np
from sklearn import linear_model
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

#df = pd.read_csv("https://media-doselect.s3.amazonaws.com/generic/dY45WBJk8BpP2jbaYJR7v2REV/training-Cars-data.zip")
df = pd.read_csv("Cars.csv")
df_imputed = df

from sklearn.impute import SimpleImputer
imp_median = SimpleImputer(missing_values = np.nan, strategy = 'median')  
df_imputed["wheelbase"] = pd.DataFrame(imp_median.fit_transform(df[["wheelbase"]].values))
df_imputed["carwidth"] = pd.DataFrame(imp_median.fit_transform(df[["carwidth"]].values))
df_imputed["carheight"] = pd.DataFrame(imp_median.fit_transform(df[["carheight"]].values))
df_imputed["enginesize"] = pd.DataFrame(imp_median.fit_transform(df[["enginesize"]].values))

df_imputed.columns = df.columns
mu = df_imputed['enginesize'].mean()
mu_r = round(mu, 3)

Correlation = round(df_imputed['enginesize'].corr(df['price']), 3)

#with open("/code/output/output1.csv", mode = "w") as f:
with open("output1.csv", mode = "w") as f:
    f.write(str(mu_r))
    f.write("\n")
    f.write(str(Correlation))


df_category = df_imputed.select_dtypes(include = ['object'])
df_category["carname"] = df_category["carname"].str.split(" ", n = 1, expand = True)
#df_category.carname.unique()
df_cat = df_category.replace(to_replace = ["maxda", "Nissan", "porcshce", "toyouta", "vokswagen", "vw"], value = ["mazda", "nissan", "porsche", "toyota", "volkswagen", "volkswagen"])
count = df_cat.carname.value_counts()['toyota']

with open("output2.csv", mode = "w") as f:
    f.write(str(count))

df_cat = df_cat["carname"].reset_index(drop = True)
df_continuous =  df_imputed.select_dtypes(include = ['float64','int64'])
df_continuous.drop(["stroke"], axis = 1, inplace = True)
df_new = pd.concat([df_continuous.reset_index(drop = True), df_cat], axis = 1)
df_getdummy = pd.get_dummies(df_new)

df_getdummy[-2:].to_csv('output3.csv', mode = "w", index = False)

x = df_getdummy.drop('price', axis = 1)
y = df_getdummy.price
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 5)
model = linear_model.LinearRegression().fit(x_train, y_train)
r2 = model.score(x_test,y_test)
r2_r = round(r2, 3)
y_intercept = model.intercept_
y_intercept_r = round(y_intercept, 3)

with open("output4.csv", mode = "w") as f:
    f.write(str(r2_r))
    f.write("\n")
    f.write(str(y_intercept_r))
    
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

ridgeReg = Ridge(alpha=0.01, normalize=True)
ridgeReg.fit(x_train,y_train)
ridge_score = ridgeReg.score(x_test,y_test)
ridge_score_r = round(ridge_score, 3)

lassoReg = Lasso(alpha=0.01)
lassoReg.fit(x_train,y_train)
lasso_score = lassoReg.score(x_test,y_test)
lasso_score_r = round(lasso_score, 3)

with open("output5.csv", mode = "w") as f:
    f.write(str(ridge_score_r))
    f.write("\n")
    f.write(str(lasso_score_r))

C:\Users\Stiff\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Stiff\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
